In [7]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
drawing = mp.solutions.drawing_utils

# OpenCV Video Capture
cap = cv2.VideoCapture(0)

# Movement tracking
total_attempts_left = 0
movement_in_progress_left = False

total_attempts_right = 0
movement_in_progress_right = False

def calculate_angle(a, b, c):
    """Calculate angle between three points."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    if angle > 180:
        angle = 360 - angle
    return angle

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # ---------- LEFT ARM ----------
        shoulder_l = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow_l = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist_l = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        angle_l = calculate_angle(shoulder_l, elbow_l, wrist_l)

        shoulder_pt_l = tuple(np.multiply(shoulder_l, [frame.shape[1], frame.shape[0]]).astype(int))
        elbow_pt_l = tuple(np.multiply(elbow_l, [frame.shape[1], frame.shape[0]]).astype(int))
        wrist_pt_l = tuple(np.multiply(wrist_l, [frame.shape[1], frame.shape[0]]).astype(int))

        if angle_l < 45:
            movement_in_progress_left = True
        elif angle_l > 120 and movement_in_progress_left:
            total_attempts_left += 1
            movement_in_progress_left = False

        color_l = (0, 255, 0) if 45 < angle_l < 120 else (0, 0, 255)

        cv2.rectangle(frame, elbow_pt_l, wrist_pt_l, color_l, 3)
        cv2.line(frame, shoulder_pt_l, elbow_pt_l, color_l, 2)
        cv2.line(frame, elbow_pt_l, wrist_pt_l, color_l, 2)
        cv2.putText(frame, f'Left Angle: {int(angle_l)}', (wrist_pt_l[0], wrist_pt_l[1] - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_l, 2)

        # ---------- RIGHT ARM ----------
        shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist_r = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        angle_r = calculate_angle(shoulder_r, elbow_r, wrist_r)

        shoulder_pt_r = tuple(np.multiply(shoulder_r, [frame.shape[1], frame.shape[0]]).astype(int))
        elbow_pt_r = tuple(np.multiply(elbow_r, [frame.shape[1], frame.shape[0]]).astype(int))
        wrist_pt_r = tuple(np.multiply(wrist_r, [frame.shape[1], frame.shape[0]]).astype(int))

        if angle_r < 45:
            movement_in_progress_right = True
        elif angle_r > 120 and movement_in_progress_right:
            total_attempts_right += 1
            movement_in_progress_right = False

        color_r = (0, 255, 0) if 45 < angle_r < 120 else (0, 0, 255)

        cv2.rectangle(frame, elbow_pt_r, wrist_pt_r, color_r, 3)
        cv2.line(frame, shoulder_pt_r, elbow_pt_r, color_r, 2)
        cv2.line(frame, elbow_pt_r, wrist_pt_r, color_r, 2)
        cv2.putText(frame, f'Right Angle: {int(angle_r)}', (wrist_pt_r[0], wrist_pt_r[1] - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_r, 2)

        drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display counters with corrected labels
    cv2.putText(frame, f'Right Arm - Attempts: {total_attempts_left}',
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 255), 2)
    cv2.putText(frame, f'Left Arm - Attempts: {total_attempts_right}',
                (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 255, 200), 2)

    cv2.imshow("Forearm Movement Tracker", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
